In [ ]:
!chmod u+x predict_all.sh

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
!./train_all.sh

Using TensorFlow backend.
***************************** FOLD 0 *****************************
Training on 3127 samples
Validating on 793 samples
['unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3_finetune_hflip', 'fold_0.hdf5']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           bn_data[0][0]                    
___________

lambda_345 (Lambda)             (None, 16, 16, 16)   0           zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
lambda_346 (Lambda)             (None, 16, 16, 16)   0           zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
lambda_347 (Lambda)             (None, 16, 16, 16)   0           zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
lambda_348 (Lambda)             (None, 16, 16, 16)   0           zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
lambda_349 (Lambda)             (None, 16, 16, 16)   0           zero_padding2d_13[0][0]          
__________________________________________________________________________________________________
lambda_350

Loading weights from /home/RnD/babakhin.y/salt/unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3/fold_0.hdf5
Epoch 1/100
157/260 [=================>............] - ETA: 2:35 - loss: 0.2845 - lb_metric: 0.9359

In [ ]:
!./train_all.sh

In [ ]:
!./train_all.sh

In [ ]:
!./evaluate_all.sh

In [ ]:
!./predict_all.sh

In [ ]:
192/224

In [ ]:
202/256

In [ ]:
320*0.8

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import cv2
import pandas as pd
import numpy as np
import os

%reload_ext autoreload
%autoreload 2

import keras
from keras import backend as K
import gc
from keras.optimizers import Adam, RMSprop, SGD

from utils import predict_test, evaluate, ensemble, ThreadsafeIter, classification_predict_test
from datasets.generators import SegmentationDataGenerator, ClassificationDataGenerator

from params import args
from callbacks import get_callback
from augmentations import get_augmentations

from models.models import get_model

from losses import *

In [ ]:

train = pd.read_csv(args.folds_csv)
test = pd.read_csv(os.path.join(args.data_root,'sample_submission.csv'))

In [ ]:
train.head()

In [ ]:
MODEL_PATH = '/home/RnD/babakhin.y/salt/unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3'

In [ ]:
from utils import evaluate
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='', snapshots=['oof'])
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))


In [ ]:
train[train.fold.isin([0])]['iout'] = res['iout']
train[train.fold.isin([0])]['dice'] = res['dice']
train[train.fold.isin([0])]['jacard'] = res['jacard']

print(train[train.fold.isin([0])].groupby('coverage_class')[['iout','dice','jacard']].aggregate('mean'))

print('Without 0 class: ',"{} / {} / {}".format(np.round(np.mean(train[train.fold.isin([0])][train[train.fold.isin([0])].coverage_class>0]['iout']),5),np.round(np.mean(train[train.fold.isin([0])][train[train.fold.isin([0])].coverage_class>0]['dice']),5),np.round(np.mean(train[train.fold.isin([0])][train[train.fold.isin([0])].coverage_class>0]['jacard']),5)))    

print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
# models = [
#     'unet_resnet_34_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_lovash_elu_1',
#     'unet_resnet_34_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_hypecolumn_csse'
#          ]
models = [
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v1',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v2',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3',
   'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005'
]


model_pathes = ['/home/RnD/babakhin.y/salt/'+x for x in models]
res = evaluate(model_pathes, train[train.fold.isin([0])].id.values, 0.5, classification='', snapshots=['oof'])
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
0.88148 / 0.91057 / 0.89055

0.48:
0.88259 / 0.91118 / 0.89119

0.46:
0.88346 / 0.91157 / 0.89158

0.45: 
0.88457 / 0.9118 / 0.89183

In [ ]:
res = evaluate([MODEL_PATH], train.id.values, 0.5, classification='', snapshots=['oof'])
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
res_list = []
for t in range(40,61):
    thr = t*0.01
    print(thr)
    res = evaluate([MODEL_PATH], train.id.values, thr, classification='')
    res_list.append(np.mean(res['iout']))

In [ ]:
res_list

In [ ]:
models = [
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005',
    'unet_resnet_152_192_224_snapshot_100_epochs_bs_16_finetune_lovash_v1'
         ]

In [ ]:
models = [
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v1',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v2',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3',
   #'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005',
    'unet_resnet_152_192_224_snapshot_100_epochs_bs_16_finetune_lovash_v1',
    #'unet_resnet_152_192_224_snapshot_100_epochs_bs_16_finetune_lovash_v1'
         ]
model_pathes = ['/home/RnD/babakhin.y/salt/'+x for x in models]
res = evaluate(model_pathes, train.id.values, 0.5, classification='', snapshots=['oof'])
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
models = [
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v1',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v2',
    'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3',
   #'unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005',
    'unet_resnet_152_192_224_snapshot_100_epochs_bs_16_finetune_lovash_v1',
    #'unet_resnet_152_192_224_snapshot_100_epochs_bs_16_finetune_lovash_v1'
         ]
model_pathes = ['/home/RnD/babakhin.y/salt/'+x for x in models]

res = evaluate(model_pathes, train.id.values, 0.41, classification='', snapshots=['oof'])
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
0.8749 / 0.90618 / 0.8861

0.48 0.87488 / 0.90648 / 0.88639

0.46: 0.87495 / 0.90743 / 0.88719
    
0.44: 0.87565 / 0.90779 / 0.88758

0.43: 0.87618 / 0.90813 / 0.88787
    
0.42: 0.87635 / 0.9081 / 0.88781
    
0.41: 0.87665 / 0.90829 / 0.88793
    
0.4: 0.8758 / 0.90815 / 0.88767

In [ ]:
MODEL_PATH

In [ ]:
model_pathes

In [ ]:
pred = ensemble(model_pathes,[0,1,2,3,4],test.id.values,0.41, classification='')
#pred = ensemble([MODEL_PATH],[0,1,2,3,4],test.id.values,0.5, classification='')
#pred = ensemble([MODEL_PATH],[0],test.id.values,0.5, classification='')
#pred = ensemble(model_pathes,[0],test.id.values,0.5, classification='')

In [ ]:
pred

In [ ]:
pred

In [ ]:
# Add older models to ensemble
# More weight for the better model

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('ens_87665_thr_041.csv',index=False)

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnext_50_lovasz_192_224_reduce_lr_smaller_decoder_bce_dice_csse_usual_00005_finetune_snapshots_low_thr_0001_v3_5_folds_87415.csv',index=False)

In [ ]:
pred = ensemble([MODEL_PATH],[2],test.id.values,0.5, classification='')
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_50_lovasz_192_224_initial_finetune_lovash_2_fold_85179.csv',index=False)

pred = ensemble([MODEL_PATH],[3],test.id.values,0.5, classification='')
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_50_lovasz_192_224_initial_finetune_lovash_3_fold_84681.csv',index=False)

pred = ensemble([MODEL_PATH],[4],test.id.values,0.5, classification='')
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_50_lovasz_192_224_initial_finetune_lovash_4_fold_84681.csv',index=False)


pred = ensemble([MODEL_PATH],[0,1,2,3,4],test.id.values,0.5, classification='')
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_50_lovasz_192_224_initial_finetune_lovash_5_folds_84786.csv',index=False)

In [ ]:
0.87037 / 0.90118 / 0.88158

In [ ]:
Ensemble 0 fold
0.8763 / 0.9064 / 0.88711

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('ensemble_0_fold_88037.csv',index=False)

In [ ]:
test.rle_mask.value_counts(normalize=True)

In [ ]:
train.rle_mask.value_counts(normalize=True,dropna=False)

In [ ]:
def plot2x2Array(image, mask):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[1].imshow(mask)
    axarr[0].grid()
    axarr[1].grid()
    axarr[0].set_title('Image')
    axarr[1].set_title('Mask')
    
for i in range(5):
    image, mask = dataset[np.random.randint(0, len(dataset))]
    plot2x2Array(image, mask)

In [ ]:
def show_results(idx):
    from rle import rle_decode
    img = rle_decode(pred[idx],(101,101))
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/images/{}.png'.format(ids_valid[idx])))
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/masks/{}.png'.format(ids_valid[idx])))
    

In [ ]:
for i in range(10):
    show_results(i)

# Classification

In [ ]:
validation_probs=[]
network = 'classification'
folds = [int(f) for f in args.fold.split(',')]
for fold in folds:
    K.clear_session()
    print('***************************** FOLD {} *****************************'.format(fold))

    MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

    if fold == 0:
        if os.path.isdir(MODEL_PATH):
            raise ValueError('Such Model already exists')
        os.system("mkdir {}".format(MODEL_PATH))
        os.system("cp params.py {}".format(MODEL_PATH))

    df_train = train[train.fold != fold].copy().reset_index(drop=True)
    df_valid = train[train.fold == fold].copy().reset_index(drop=True)

    ids_train, ids_valid = df_train.id.values, df_valid.id.values

    print('Training on {} samples'.format(ids_train.shape[0]))
    print('Validating on {} samples'.format(ids_valid.shape[0]))

    # Initialize Model
    weights_path = os.path.join(MODEL_PATH,'fold_{fold}.hdf5'.format(fold=fold))

    print(weights_path.split('/')[-2:])

    model, preprocess = get_model(network, input_shape=(args.input_size, args.input_size, 3), train_base=True)
    print(model.summary())
    model.compile(optimizer=RMSprop(lr=args.learning_rate), loss='binary_crossentropy',
                      metrics=['accuracy'])

    if args.weights is None:
        print('No weights passed, training from scratch')
    else:
        wp = args.weights.format(fold)
        print('Loading weights from {}'.format(wp))
        model.load_weights(wp, by_name=True)

    augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))


    dg = ClassificationDataGenerator(input_shape=(args.input_size, args.input_size), batch_size = args.batch_size, augs = augs,
                      preprocess = preprocess)

    train_generator = dg.train_batch_generator(ids_train)
    validation_generator = dg.evaluation_batch_generator(ids_valid)

    callbacks = get_callback(args.callback, weights_path=weights_path,
                            fold = fold)

    # Fit the model with Generators:
    model.fit_generator(generator=ThreadsafeIter(train_generator),
                    steps_per_epoch=ids_train.shape[0] // args.batch_size * 2,
                    epochs=args.epochs,
                    callbacks=callbacks,
                    validation_data=ThreadsafeIter(validation_generator),
                    validation_steps=np.ceil(ids_valid.shape[0] / args.batch_size),
                    workers=8)

    model.load_weights(weights_path)

    # SAVE OOF PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'oof')
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=True,
                    ids=ids_valid,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    df_valid['prob'] = pred
    validation_probs.append(df_valid)

    # SAVE TEST PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'fold_{}'.format(fold))
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=False,
                    ids=test.id.values,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    test['prob'] = pred
    test.to_csv(os.path.join(dir_path,'probs_test_fold_{}.csv'.format(fold)),index=False)

    K.clear_session()
    # Run a single fold
    # break

dir_path = os.path.join(MODEL_PATH,'oof')
pd.concat(validation_probs).to_csv(os.path.join(dir_path,'probs_oof.csv'),index=False)